Scalable Agentic AI Video Classification with LoRA Fine-Tuning, LangGraph-Based and NLP Video Summarization
SECTION 1: INSTALLATION AND SETUP

In [1]:
# Install required packages
!pip install -q langgraph typing_extensions torch torchvision peft
!pip install -q moviepy SpeechRecognition pydub
!apt-get install -y ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 932.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.3/469.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 7.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 37.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [2]:
import os
import glob
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import resnet18
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from peft import LoraConfig, get_peft_model
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip, concatenate_videoclips
import speech_recognition as sr
from pydub import AudioSegment
from skimage.metrics import structural_similarity as ssim
from IPython.display import Video, display
from google.colab import drive

# LangGraph Imports
from typing import TypedDict
from langgraph.graph import StateGraph, END

/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':

  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)

  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)

  elif re.match('(flt)p?( \(default\))?$', token):

  elif re.m

In [3]:
# Mount Google Drive
drive.mount('/content/drive')

# Define paths
zip_file_path_in_drive = '/content/drive/My Drive/aai-521/videos-ds.zip'
destination_folder = '/content/drive/My Drive/aai-521/videos-ds'
train_data_dir = os.path.join(destination_folder, 'train/')
test_data_dir = os.path.join(destination_folder, 'test/')
train_csv_path = os.path.join(destination_folder, 'train.csv')
test_csv_path = os.path.join(destination_folder, 'test.csv')

# Create destination folder if needed
os.makedirs(destination_folder, exist_ok=True)

# Copy and unzip data
if os.path.exists(zip_file_path_in_drive):
    !cp "{zip_file_path_in_drive}" "{destination_folder}/videos-ds.zip"
    print(" videos-ds.zip loaded from Google Drive")

    zip_file_path = os.path.join(destination_folder, "videos-ds.zip")
    !unzip -o -q "{zip_file_path}" -d "{destination_folder}"
    print(" Data extracted successfully")

    # Verify files
    avi_files = glob.glob(os.path.join(destination_folder, '**', '*.avi'), recursive=True)
    print(f" Found {len(avi_files)} video files")
else:
    print("Error: videos-ds.zip not found")

Mounted at /content/drive
 videos-ds.zip loaded from Google Drive
 Data extracted successfully
 Found 818 video files


In [4]:
def extract_frames(video_path, num_frames=16):
    """Extract evenly-spaced frames from video, padding if necessary."""
    frames = []
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return [None] * num_frames # Return list of None if video cannot be opened

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames == 0:
        cap.release()
        return [None] * num_frames # Return list of None if video has no frames

    frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)

    extracted_count = 0
    for i in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
            extracted_count += 1
        else:
            frames.append(None) # Append None if frame extraction fails

    cap.release()

    # Pad with None if fewer than num_frames were extracted
    while len(frames) < num_frames:
        frames.append(None)

    return frames


def collate_fn(batch):
    """Custom collate function to filter None values and ensure consistent frame count."""
    # Filter out None items from the batch (returned by __getitem__ for videos with no valid frames)
    batch = [item for item in batch if item is not None]

    if not batch:
        return None, None

    inputs, labels_or_paths = zip(*batch)

    # inputs is now a tuple of tensors (Batch x Frames x Channels x Height x Width)
    # Stack the tensors along the batch dimension
    return torch.stack(inputs), list(labels_or_paths)

In [5]:
class VideoDataset(Dataset):
    """Custom dataset for video classification."""

    def __init__(self, data_dir, csv_path, num_frames=16, transform=None, is_test=False):
        self.data_dir = data_dir
        self.transform = transform
        self.videos = []
        self.labels = []
        self.is_test = is_test
        self.video_paths = []
        self.num_frames = num_frames # Store num_frames

        df = pd.read_csv(csv_path)

        for _, row in df.iterrows():
            video_name = row['video_name']
            if not self.is_test:
                self.labels.append(row['tag'])

            video_path = os.path.join(data_dir, video_name)
            if os.path.exists(video_path):
                self.videos.append(video_path)
                if self.is_test:
                    self.video_paths.append(video_path)

    def __len__(self):
        return len(self.videos)

    def __getitem__(self, idx):
        video_path = self.videos[idx]
        frames = extract_frames(video_path, self.num_frames) # Pass num_frames

        if not frames or all(f is None for f in frames):
            return None # Return None if no frames could be extracted

        processed_frames = []
        for frame in frames:
            if frame is not None:
                # Convert frame (numpy array) to tensor and apply transform
                frame_tensor = torch.from_numpy(frame).permute(2, 0, 1).float() # HxWx3 to 3xHxW
                if self.transform:
                    processed_frames.append(self.transform(frame_tensor)) # Apply transform
                else:
                    processed_frames.append(frame_tensor)
            else:
                 # Append a tensor of zeros for None frames
                 # Assuming frames are 3 channels, H=112, W=112 after transform
                 processed_frames.append(torch.zeros(3, 112, 112))


        # Stack the processed frames for the current video
        stacked_frames = torch.stack(processed_frames)

        if not self.is_test:
            return stacked_frames, self.labels[idx]
        else:
            return stacked_frames, video_path

In [6]:
# Define transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = VideoDataset(train_data_dir, train_csv_path, transform=transform, is_test=False)
test_dataset = VideoDataset(test_data_dir, test_csv_path, transform=transform, is_test=True)

# Create dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)

print(f"Training samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")
print(f"Number of classes: {len(set(train_dataset.labels))}")

Training samples: 594
Test samples: 224
Number of classes: 5


In [8]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f" Using device: {device}")

# Load pre-trained ResNet-18
num_classes = len(set(train_dataset.labels))
model = resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.to(device)

# Apply LoRA
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "conv1", "layer1.0.conv1", "layer1.0.conv2", "layer1.1.conv1", "layer1.1.conv2",
        "layer2.0.conv1", "layer2.0.conv2", "layer2.0.downsample.0", "layer2.1.conv1", "layer2.1.conv2",
        "layer3.0.conv1", "layer3.0.conv2", "layer3.0.downsample.0", "layer3.1.conv1", "layer3.1.conv2",
        "layer4.0.conv1", "layer4.0.conv2", "layer4.0.downsample.0", "layer4.1.conv1", "layer4.1.conv2",
        "fc"
    ],
    lora_dropout=0.05,
    bias="none"
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

# Optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

print(" Model configured with LoRA")

 Using device: cpu
trainable params: 583,040 || all params: 11,762,117 || trainable%: 4.9569
 Model configured with LoRA


In [9]:
num_epochs = 1
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_dataloader):
        if data is None or data[0] is None:
            continue

        inputs, labels = data
        unique_labels = sorted(list(set(train_dataset.labels)))
        label_to_int = {label: idx for idx, label in enumerate(unique_labels)}
        labels = torch.tensor([label_to_int[l] for l in labels], dtype=torch.long).to(device)
        inputs = inputs.to(device)

        optimizer.zero_grad()
        # Treat all frames in the batch as individual images for the 2D CNN (ResNet)
        outputs = model(inputs.view(-1, 3, 112, 112))
        # Average the frame-level predictions to get a video-level prediction
        outputs = outputs.view(inputs.size(0), inputs.size(1), -1).mean(dim=1)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if (i + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}], Loss: {running_loss/10:.4f}')
            running_loss = 0.0

Epoch [1/1], Step [10], Loss: 1.3486
Epoch [1/1], Step [20], Loss: 0.3511
Epoch [1/1], Step [30], Loss: 0.3536


In [10]:
print("\n" + "="*80)
print("VIDEO CLASSIFICATION EVALUATION")
print("="*80)

model.eval()
predictions = []
video_paths = []

with torch.no_grad():
    for data in test_dataloader:
        if data is None or data[0] is None:
            continue

        inputs, paths = data
        # Use torch.stack to ensure consistent tensor structure if collate_fn returned a list of tensors
        # inputs = torch.stack(inputs).to(device) # Removed redundant stack
        inputs = inputs.to(device)

        outputs = model(inputs.view(-1, 3, 112, 112))
        outputs = outputs.view(inputs.size(0), inputs.size(1), -1).mean(dim=1)

        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())
        video_paths.extend(paths)

# Map predictions to labels
unique_labels = sorted(list(set(train_dataset.labels)))
label_map = {i: label for i, label in enumerate(unique_labels)}
predicted_labels = [label_map[pred] for pred in predictions]

results_df = pd.DataFrame({
    'video_path': video_paths,
    'predicted_label': predicted_labels
})

print("\n Predictions on test videos:")
print(results_df.head(10))


VIDEO CLASSIFICATION EVALUATION

 Predictions on test videos:
                                          video_path predicted_label
0  /content/drive/My Drive/aai-521/videos-ds/test...     TennisSwing
1  /content/drive/My Drive/aai-521/videos-ds/test...     TennisSwing
2  /content/drive/My Drive/aai-521/videos-ds/test...     CricketShot
3  /content/drive/My Drive/aai-521/videos-ds/test...     CricketShot
4  /content/drive/My Drive/aai-521/videos-ds/test...     CricketShot
5  /content/drive/My Drive/aai-521/videos-ds/test...     CricketShot
6  /content/drive/My Drive/aai-521/videos-ds/test...     CricketShot
7  /content/drive/My Drive/aai-521/videos-ds/test...    PlayingCello
8  /content/drive/My Drive/aai-521/videos-ds/test...    PlayingCello
9  /content/drive/My Drive/aai-521/videos-ds/test...    PlayingCello


In [11]:
!pip install pygooglenews

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 6.1 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=ecd87efffcf0b50614b74b7057935e38ad51248fe3c8b2dd6e38b1aab6f50418
  Stored in directory: /root/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [ ]:
import nltk

# Download necessary NLTK data
try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')

try:
    nltk.data.find('corpora/omw-1.4')
except LookupError:
    nltk.download('omw-1.4')

print("NLTK data downloaded successfully.")

NLTK data downloaded successfully.


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [28]:
# Define the State of the LangGraph Agent
from typing import TypedDict, List # Import List here
from pygooglenews import GoogleNews
from IPython.display import Video, display
import cv2
import numpy as np
import pandas as pd
from moviepy.editor import VideoFileClip, concatenate_videoclips
import speech_recognition as sr
from pydub import AudioSegment
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import re


class SummaryState(TypedDict):
    video_path: str
    audio_path: str
    summary_video_path: str
    transcript: str
    duration: float
    summary_clips_count: int
    error: str
    # --- NEW STATE KEY ---
    news: List[dict] # To store fetched news articles (title, link, source)
    predicted_label: str # Add predicted label to the state
    processed_label_words: List[str] # Add key for processed label words

# --- 10.1 LangGraph Nodes (Functions) ---

def extract_audio(state: SummaryState) -> SummaryState:
    """Node 1: Extracts audio from the video and updates the state."""
    print("\n 1. Executing Audio Extraction...")
    video_path = state['video_path']
    audio_path = video_path.replace('.avi', '.wav')

    try:
        video_clip = VideoFileClip(video_path)
        video_clip.audio.write_audiofile(audio_path, logger=None)
        video_clip.close()
        print("✓ Audio extracted.")
        return {**state, "audio_path": audio_path, "error": None}
    except Exception as e:
        print(f"✗ Audio extraction failed: {e}")
        return {**state, "error": f"Audio extraction failed: {e}"}


def transcribe_audio(state: SummaryState) -> SummaryState:
    """Node 2: Transcribes the extracted audio and updates the state with the transcript."""
    print(" 2. Executing Transcription...")
    audio_path = state.get('audio_path')

    if not audio_path or state.get('error'):
        return {**state, "transcript": "Skipped due to prior error.", "error": state.get('error')}

    transcript = "No speech detected in video"
    try:
        r = sr.Recognizer()
        with sr.AudioFile(audio_path) as source:
            audio_data = r.record(source)
            transcript = r.recognize_google(audio_data)
        print(f"✓ Transcript generated: {transcript[:50]}...")
        return {**state, "transcript": transcript, "error": None}
    except Exception as e:
        print(f"⚠ Transcription failed: {e} (assuming no speech)")
        return {**state, "transcript": transcript, "error": None}


def create_summary_video(state: SummaryState) -> SummaryState:
    """Node 3: Extracts key frames and creates the summary video."""
    print(" 3. Executing Summary Video Creation...")
    video_path = state['video_path']
    summary_video_path = video_path.replace('.avi', '_summary.mp4')

    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    cap.release()

    if fps == 0:
        return {**state, "error": "FPS is zero, cannot calculate duration/frames."}

    duration = frame_count / fps
    frame_indices = list(range(0, frame_count, 15)) # Extract frame every 15 frames
    segment_duration = 1 # 1 second clip per key frame
    summary_clips = []

    try:
        for frame_idx in frame_indices:
            start_time = frame_idx / fps
            end_time = min(start_time + segment_duration, duration)

            if end_time > start_time:
                clip = VideoFileClip(video_path).subclip(start_time, end_time)
                summary_clips.append(clip)

        if summary_clips:
            final_summary = concatenate_videoclips(summary_clips)
            final_summary.write_videofile(summary_video_path, codec='libx264',
                                           audio_codec='aac', logger=None)
            final_summary.close()
            print(f"✓ Summary video saved: {summary_video_path}")

            return {**state,
                    "summary_video_path": summary_video_path,
                    "duration": duration,
                    "summary_clips_count": len(summary_clips),
                    "error": None
                   }
        else:
            return {**state, "error": "No clips extracted for summary."}
    except Exception as e:
        return {**state, "error": f"Summary creation failed: {e}"}


# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize_label(state: SummaryState) -> SummaryState:
    """Node: Processes the predicted label into lemmatized words and filters irrelevant terms."""
    print("\n 4. Executing Label Lemmatization...")
    predicted_label = state.get('predicted_label')

    if not predicted_label:
        print("--- Node: lemmatize_label ---")
        print("Error: Predicted label not found. Skipping lemmatization.")
        return {**state, "processed_label_words": []}

    print(f"--- Node: lemmatize_label (Original Label: {predicted_label}) ---")

    # Convert CamelCase to separate words
    words = re.findall('[A-Z][a-z]*', predicted_label)
    if not words: # Handle cases like "Punch" which might not be split by CamelCase
        words = [predicted_label]

    lemmatized_words = [lemmatizer.lemmatize(word.lower(), pos=wordnet.VERB) for word in words] # Lemmatize as verbs initially
    lemmatized_words = [lemmatizer.lemmatize(word, pos=wordnet.NOUN) for word in lemmatized_words] # Then lemmatize as nouns

    # Simple filtering: remove common verbs that might not be the primary action
    # This list can be expanded based on the specific labels in your dataset
    irrelevant_verbs = ['shoot', 'play', 'swing']
    filtered_words = [word for word in lemmatized_words if word not in irrelevant_verbs]

    # Further filtering: keep only words that are likely nouns or verbs representing the core concept
    # This is a more advanced approach and might require more sophisticated NLP techniques
    # For now, we'll stick to the simple irrelevant verb filtering

    print(f"✓ Processed Label Words: {filtered_words}")

    return {**state, "processed_label_words": filtered_words}


def fetch_news_node(state: SummaryState) -> dict:
    """
    Fetches news from Google News based on the video's classified topic.
    """
    gn = GoogleNews(lang='en', country='US')

    # 1. Extract the search query (classified topic) - now use processed words
    topic_words = state.get("processed_label_words")

    # 2. Handle cases where the topic is missing
    if not topic_words:
        print("--- Node: fetch_news ---")
        print("Error: Processed label words not found. Skipping news fetch.")
        return {"news": []}

    # Join the words to form a search query
    search_query = " ".join(topic_words)
    print(f"--- Node: fetch_news (Search Query: {search_query}) ---")

    try:
        # 3. Search Google News (using pygooglenews wrapper)
        # Search for articles about the topic in the last 7 days ('7d') for freshness
        search_result = gn.search(query=search_query, when='7d')

        fetched_news = []
        # 4. Process and structure the results
        for entry in search_result['entries']:
            # The structure of the entry is provided by the RSS feed/pygooglenews (Source 2.6)
            fetched_news.append({
                "title": entry.title,
                "link": entry.link,
                "published": entry.published,
                "source": entry.source.get('title', 'Unknown Source')
            })

        print(f"Successfully fetched {len(fetched_news)} news articles.")

        # 5. Return the update to the LangGraph state
        return {"news": fetched_news}

    except Exception as e:
        print(f"An error occurred while fetching news: {e}")
        return {"news": []}

def display_news_node(state: SummaryState) -> SummaryState:
    """Node: Displays the top 5 news articles."""
    print("\n 5. Executing Display News...")
    news_articles = state.get('news')

    print("\n" + "="*80)
    print("TOP 5 RELATED NEWS ARTICLES")
    print("="*80)

    if news_articles:
        for i, article in enumerate(news_articles[:5]):
            print(f"{i+1}. Title: {article.get('title', 'N/A')}")
            print(f"   Link: {article.get('link', 'N/A')}")
            print("-" * 20)
    else:
        print("No news articles were fetched.")

    print("="*80)

    return state # Return the state unchanged

# --- 10.2 Build and Run the LangGraph ---

# Build the graph
workflow = StateGraph(SummaryState)

# Add nodes
workflow.add_node("extract_audio", extract_audio)
workflow.add_node("transcribe_audio", transcribe_audio)
workflow.add_node("create_summary_video", create_summary_video)
workflow.add_node("lemmatize_label", lemmatize_label) # Add the new lemmatization node
workflow.add_node("fetch_news_node", fetch_news_node)
workflow.add_node("display_news_node", display_news_node) # Add the display news node

# Set edges
workflow.add_edge('extract_audio', 'transcribe_audio')
workflow.add_edge('transcribe_audio', 'create_summary_video')
workflow.add_edge('create_summary_video', 'lemmatize_label') # Add edge to lemmatization node
workflow.add_edge('lemmatize_label', "fetch_news_node") # Add edge from lemmatization node to fetch_news_node
workflow.add_edge("fetch_news_node", "display_news_node") # Add edge from fetch news to display news
workflow.add_edge("display_news_node", END) # Add edge from display news to END

# Set the starting point
workflow.set_entry_point('extract_audio')

# Compile the graph
app = workflow.compile()

# Run the graph on a sample video
print("\n" + "="*80)
print("LANGGRAPH EXECUTION START")
print("="*80)

sample_video_path = '/content/drive/My Drive/aai-521/videos-ds/test/v_CricketShot_g01_c03.avi'

# Get the predicted label for the sample video from results_df
sample_video_row = results_df[results_df['video_path'] == sample_video_path]
if not sample_video_row.empty:
    predicted_label = sample_video_row['predicted_label'].iloc[0]
else:
    predicted_label = "Unknown" # Default if not found

initial_state = {
    'video_path': sample_video_path,
    'audio_path': '',
    'summary_video_path': '',
    'transcript': '',
    'duration': 0.0,
    'summary_clips_count': 0,
    'error': None,
    'predicted_label': predicted_label, # Add the predicted label to the initial state
    'processed_label_words': [] # Initialize processed label words
}

final_state = app.invoke(initial_state)

# --- 10.3 Display Final Results ---

print("\n" + "="*80)
print("LANGGRAPH FINAL SUMMARY OUTPUT")
print("="*80)

if final_state.get('error'):
    print(f"ERROR: {final_state['error']}")
else:
    print(f"Video Path: {final_state['video_path']}")
    print(f"Predicted Label: {final_state.get('predicted_label', 'N/A')}") # Display original predicted label
    print(f"Processed Label Words: {final_state.get('processed_label_words', 'N/A')}") # Display processed words
    print(f"Transcript: {final_state['transcript']}")
    print(f"Original duration: {final_state['duration']:.2f}s")
    print(f"Summary duration: {final_state['summary_clips_count']}s")
    print(f"News Articles: {len(final_state.get('news', []))}") # Display number of fetched news articles

    try:
        display(Video(final_state['summary_video_path'], embed=True, width=640))
    except:
        print(f"Video saved but cannot display inline: {final_state['summary_video_path']}")

print("\n" + "="*80)
print("PROCESSING COMPLETE")
print("="*80)


LANGGRAPH EXECUTION START

 1. Executing Audio Extraction...
✓ Audio extracted.
 2. Executing Transcription...
⚠ Transcription failed:  (assuming no speech)
 3. Executing Summary Video Creation...
✓ Summary video saved: /content/drive/My Drive/aai-521/videos-ds/test/v_CricketShot_g01_c03_summary.mp4

 4. Executing Label Lemmatization...
--- Node: lemmatize_label (Original Label: CricketShot) ---
✓ Processed Label Words: ['cricket']
--- Node: fetch_news (Search Query: cricket) ---
Successfully fetched 100 news articles.

 5. Executing Display News...

TOP 5 RELATED NEWS ARTICLES
1. Title: India beat South Africa by 52 runs to win Women’s Cricket World Cup final – live reaction - The Guardian
   Link: https://news.google.com/rss/articles/CBMiqwFBVV95cUxObEtrV3BnZ1RwM1BRZE5EVDlEQ25oamxxcWVMVnhKdHlFcUROaEZnUW9id2JGTXNRMnFnLXpNbkNYdURQeXVNcVU4S2p3NWZYZXJZb0lpWFNmSlRMRzVrR1Nqd3N4MFc2dC1fbURVMUNTd0xhM3pWcWNyeU5OM1hPV3hPR016UGxjODJxUXJoUXBZNXFCdU0wWEZwS09IQ2xZUUlxU09uS3loSUk?oc=5
------------


PROCESSING COMPLETE
